<h1>SparkSQL<h1>

<h2>Importing libraries<h2>

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
spark=SparkSession.builder.master('local[*]').getOrCreate()
sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)
df_ratings=sqlCtx.read.format("jdbc").options(url ="jdbc:sqlite:movielens-small.db", driver="org.sqlite.JDBC", dbtable="ratings").load()
df_movies=sqlCtx.read.format("jdbc").options(url ="jdbc:sqlite:movielens-small.db", driver="org.sqlite.JDBC", dbtable="movies").load()
df_links=sqlCtx.read.format("jdbc").options(url ="jdbc:sqlite:movielens-small.db", driver="org.sqlite.JDBC", dbtable="links").load()
df_tags=sqlCtx.read.format("jdbc").options(url ="jdbc:sqlite:movielens-small.db", driver="org.sqlite.JDBC", dbtable="tags").load()





<h2>Table Schemas<h2>

In [2]:
df_ratings.printSchema()
df_movies.printSchema()
df_links.printSchema()
df_tags.printSchema()

root
 |-- userId: long (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

root
 |-- movieId: long (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- movieId: long (nullable = true)
 |-- imdbId: string (nullable = true)
 |-- tmdbId: string (nullable = true)

root
 |-- userId: long (nullable = true)
 |-- movieId: long (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: decimal(38,18) (nullable = true)



<h2>Creating Table Views<h2>

In [3]:
df_ratings.createOrReplaceTempView("df_ratings")
df_movies.createOrReplaceTempView("df_movies")
df_links.createOrReplaceTempView("df_links")
df_tags.createOrReplaceTempView("df_tags")



<h2>Selecting all tables from our movie db<h2>

In [4]:
df_ratings= spark.sql("SELECT * from df_ratings")
df_movies= spark.sql("SELECT * from df_movies")
df_links= spark.sql("SELECT * from df_links")
df_tags= spark.sql("SELECT * from df_tags")



<h2>Creating the main table with movies and ratings table<h2>

In [5]:
df_main=spark.sql("select t2.userId,t1.movieId,t1.genres,t2.rating  from df_movies t1 join df_ratings t2 on t1.movieId=t2.movieId")

<h2>Countings ratings for each movie,and listing top 5<h2>

In [6]:

df_main.createOrReplaceTempView("df_second")
df_top_five_movies= spark.sql("SELECT count(rating) as cnt_rating,movieId from df_second group by movieId order by cnt_rating desc LIMIT 5 ")


<h2>Preview of our df_top_five_movies table<h2>

In [7]:
df_top_five_movies.show()


+----------+-------+
|cnt_rating|movieId|
+----------+-------+
|       337|    593|
|       328|    318|
|       327|    296|
|       324|    480|
|       318|    356|
+----------+-------+



<h2>Finding and listing top 5 most rated genres <h2>

In [8]:

df_main.createOrReplaceTempView("df_main_2")
df_top_genres= spark.sql("SELECT * from df_main_2 ")
df_top_genres_sol= spark.sql("SELECT movieId,round(avg(rating),2) as rating_avg from df_main_2 group by movieId order by rating_avg desc LIMIT 5 ")
df_top_genres_sol.show()



+-------+----------+
|movieId|rating_avg|
+-------+----------+
|  63836|       5.0|
|   7234|       5.0|
|  33585|       5.0|
|  87992|       5.0|
|  57910|       5.0|
+-------+----------+

